In [4]:
import json
from pathlib import Path
from datetime import date
import base64
from getpass import getpass
import mimetypes


from IPython.display import Image
import instructor
from pydantic import BaseModel
from openai import OpenAI
import pandas as pd

In [5]:
#Wprowadz klucz OPENAI
openai_key = getpass("Wprowadz swoj klucz OPENAI:")
openai_client = OpenAI(api_key=openai_key)


Wprowadz swoj klucz OPENAI: ········


In [40]:
# Promompty
TABLE_PROMPT = """
Z podanego screeenshota w postaci screena z tabeli chce abys przeksztalcil to na dane tekstowe w postaci pythonowego slownika.
Kluczem bedzie data np "3/22" i reszte danych musisz przedstawic w formie jaka uznasz za stosowne.

Przykladowy output to:

{
    "3/22": {
        "Matchup": "DAL",
        "Score": "L 123-117",
        "Spread Result": "DAL +9.5",
        "Minutes": 27,
        "Prop Line": 13.5,
        "PTS": "U 8",
        "REBS": 9,
        "ASSISTS": 3,
        "3s Scored": 0,
        "STEALS": 2,
        "BLOCKS": 2,
        "FGM-FGA (%)": "5-5 (100%)",
        "FTM-FTA (%)": "4-6 (67%)",
        "3PM-3PA (%)": "0-0 (0%)"
    }
}

Jako koncowy format zwrocony oczekuje Liste Slownikow z takimi wlasnie wartosciami jak tez z przykladu.
Kazdy wiersz to ma byc osobny slownik dodany do listy. 

PAMIETAJ ZEBY NIE DODAWAC ZADNYCH KOMENTARZY TYLKO DANE O KTORE PROSZE

"""


In [8]:
# RAW_IMAGE_FOLDER = Path("Images")
# PROCESSED_IMAGES = RAW_IMAGE_FOLDER / "Processed"

In [9]:
# for image_path in RAW_IMAGE_FOLDER.glob("*.png"):
#     print(image_path)

In [26]:
from IPython.display import Image, display
image_path = "local_storage/Jalen_Duren.png"



In [16]:
# Przygotowanie pliku do przeslania go do OpenAI - base64



def prep_file_for_openai(image_path):
    # Sprawdzenie MIME typu na podstawie rozszerzenia pliku
    mime_type, _ = mimetypes.guess_type(image_path)
    
    # Obsługiwane typy obrazów
    supported_types = ["image/png", "image/jpeg"]
    
    # Sprawdzenie, czy typ MIME jest obsługiwany
    if mime_type not in supported_types:
        raise ValueError(f"Nieobsługiwany format obrazu: {mime_type}. Obsługiwane formaty to: {', '.join(supported_types)}")
    
    # Wczytanie obrazu i zakodowanie go w Base64
    with open(image_path, 'rb') as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
        return f"data:{mime_type};base64,{image_data}"

encoded_file = prep_file_for_openai(image_path=image_path)
print(encoded_file[:100])

data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAABUMAAA8JCAYAAADuFk3SAAAEDmlDQ1BrQ0dDb2xvclNwYWNlR2VuZX


In [32]:
def generate_ai_response(image_path):

    response = openai_client.chat.completions.create(
        # model="gpt-4o",
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": TABLE_PROMPT
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": prep_file_for_openai(image_path=image_path),
                            "detail": "high"
                        },
                    },
                ],
            }
        ],
    )


    return (response.choices[0].message.content)

In [33]:
supported_extensions = ["*.png", "*.jpg", "*.jpeg"]

output = generate_ai_response(image_path)


In [35]:
folder_path = Path("images")

In [41]:
lista = []
for image_path in folder_path.glob("*.png"):  # Możesz użyć np. "*.png" dla PNG
    print(image_path)
    output = generate_ai_response(image_path)
    lista.append(output)
    print(output)

images/SCR-20250323-rruw.png
images/SCR-20250323-rsfs.png
images/SCR-20250323-rsme.png
images/SCR-20250323-rsuc.png
images/SCR-20250323-rsby.png
images/SCR-20250323-rthd.png
images/SCR-20250323-rsos.png
images/SCR-20250323-rtjh.png
images/SCR-20250323-rtdt.png
images/SCR-20250323-rsyx.png
images/SCR-20250323-rsrx.png
images/SCR-20250323-rsjh.png
images/SCR-20250323-rtbq.png
images/SCR-20250323-rrzf.png


In [46]:
random_image = "images/SCR-20250323-rruw.png"
output = generate_ai_response(image_path)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}